### Check out Cell 3 for Columns that are dropped
### For Feature engineering, below are the new or updated features:
1. 'large_family' --> If People HH > 3 then 1; 0 otherwise
2. 'MaritalStatus' --> updated to 'Married', 'Single', 'Separated/Divorced', 'Unknown'
3. 'USStayLength' --> updated to 'Unknown', 'Short Term (if less <= 10 years)', 'Long Term (>10 years)'
4. 'FamilyIncome' --> updated to 'Low' (if <= 55k), 'Medium' (between 55k to 100k), 'High'(>100k) based on US baseline
5. 'college_educated' --> 1 if college educated, 0 if not college educated, 2 if unknown
6. 'Citizenship' --> 0 if not US citizen, 1 if US citizen
7. 'high_risk' (diabetic potential) --> 1 if BMI is above or equal to 25, Two Hour Glucose(OGTT) (mg/dL) >= 140 and 'Glycohemoglobin (%)' >= 5.7; 0 otherwise

Note that 'PeopleHH' and 'EduLevelAdult' are dropped after getting replaced by 1. and 5.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
df_final = pd.read_csv('merged_raw.csv', index_col = [0])
df_final.dropna(inplace=True)

In [3]:
col_to_drop = ['Amount of glucose challenge drank',
'Total length of "food fast," hours',
'Total # of Antacids Taken',
'Any Antacids Taken?',
'Blood Pressure Status',
'Recumbent Length (cm)',
'Head Circumference (cm)',
'Weight (kg)',
'Sagittal Abdominal Diameter 1st (cm)',
'Grip strength (kg), hand 1, test 1',
'Grip strength (kg), hand 2, test 1',
'Dominant hand', 
'Testing position',
'Grip strength (kg), hand 1, test 1',
'Grip strength (kg), hand 2, test 1',
'Salmon eaten during past 30 days',
'Coronal Caries: Surface condition #2',
'Dental Sealants: Upper right 2nd molar (2M) sealant codes',
'Coronal Caries: Tooth count #2',
'Is_Diabetic', 
'final_IsDia',
'Unnamed: 0.1',
'BirthCountry']

df_final.drop(columns=col_to_drop, inplace = True)

In [5]:
df_final.loc[df_final['PeopleHH'] > 3 , 'large_family'] = 1
df_final.loc[df_final['PeopleHH'] <= 3 , 'large_family'] = 0
df_final.loc[~df_final['EduLevelAdult'].isin(['College/AA', 'College/Above', '>HighSchool']) , 'college_educated'] = 0
df_final.loc[df_final['EduLevelAdult'].isin(['College/AA', 'College/Above', '>HighSchool']) , 'college_educated'] = 1
df_final.loc[df_final['EduLevelAdult'].isin(['Unknown', 'DontKnow']) , 'college_educated'] = 2
df_final.loc[df_final['FamilyIncome'].isin(['<5k','5-10k', '10-15k','15-20k', '20-25k', '25-35k','35-45k', '45-55k']) , 'FamilyIncome'] = 'Low'
df_final.loc[df_final['FamilyIncome'].isin(['55-65k','65-75k', '75-99k']) , 'FamilyIncome'] = 'Medium'
df_final.loc[df_final['FamilyIncome'].isin(['>100k']) , 'FamilyIncome'] = 'High'

In [6]:
df_final.loc[df_final['USStayLength'].isin(['Unknown','DontKnow', 'Refused']) , 'USStayLength'] = 'Unknown'
df_final.loc[df_final['USStayLength'].isin(['<1', '1-5', '5-10']) , 'USStayLength'] = 'Short Term (< 10 Years)'
df_final.loc[df_final['USStayLength'].isin(['10-15', '15-20','20-30', '30-40','40-50', '>50']) , 'USStayLength'] = 'Long Term (> 10 Years)'

In [7]:
df_final.loc[df_final['Citizenship'] != 'USCitizen', 'Citizenship'] = 0
df_final.loc[df_final['Citizenship'] == 'USCitizen', 'Citizenship'] = 1

In [8]:
changed_col = ['PeopleHH', 'EduLevelAdult']
df_final.drop(columns=changed_col, inplace = True)

In [9]:
df_final.loc[df_final['MaritalStatus'].isin(['Unknown','DontKnow', 'Refused']) , 'MaritalStatus'] = 'Unknown'
df_final.loc[df_final['MaritalStatus'].isin(['Divorced', 'Separated', 'Widowed']) , 'MaritalStatus'] = 'Separated/Divorced'
df_final.loc[df_final['MaritalStatus'].isin(['Married', 'LivingIn']) , 'MaritalStatus'] = 'Married'
df_final.loc[df_final['MaritalStatus'].isin(['NeverMarried']) , 'MaritalStatus'] = 'Single'

In [15]:
df_final['high_risk'] = 0
df_final.loc[(df_final['AgeYears'] <= 20) & (df_final['Body Mass Index (kg/m**2)'] >= 25) & (df_final['Glycohemoglobin (%)'] >= 5.7) & (df_final['Two Hour Glucose(OGTT) (mg/dL)'] >= 140), 'high_risk'] = 1
df_final.loc[(df_final['AgeYears'] > 20) & (df_final['AgeYears'] <= 40) & (df_final['Body Mass Index (kg/m**2)'] >= 25) & (df_final['Glycohemoglobin (%)'] >= 5.7) & (df_final['Two Hour Glucose(OGTT) (mg/dL)'] >= 120), 'high_risk'] = 1
df_final.loc[(df_final['AgeYears'] > 40) & (df_final['Body Mass Index (kg/m**2)'] >= 25) & (df_final['Glycohemoglobin (%)'] >= 5.7) & (df_final['Two Hour Glucose(OGTT) (mg/dL)'] >= 100), 'high_risk'] = 1

In [17]:
df_final[(df_final['high_risk'] > 0) & (df_final['is_diabetic'] > 0)]

,SEQN,Cadmium (ug/L),Lead (ug/dL),Blood manganese (ug/L),"Mercury, total (ug/L)",Blood selenium(ug/L),LDL-Cholesterol,Lymphocyte number,Monocyte number,Segmented neutrophils number,...,Citizenship,USStayLength,MaritalStatus,PregnancyStatus,FamilyIncome,PovertyRatio,is_diabetic,large_family,college_educated,high_risk
0,73559,0.22,1.45,9.57,1.27,209.64,126.00,1.00,0.80,4.90,...,1,Unknown,Married,NotApplicable,Medium,4.51,1,0.0,1.0,1
7,73659,0.14,0.97,16.48,0.73,183.18,241.00,3.20,0.60,5.50,...,1,Unknown,Separated/Divorced,Unknown,Low,1.93,1,0.0,1.0,1
13,73694,0.29,1.30,8.83,0.71,244.29,191.00,1.50,0.70,2.10,...,1,Unknown,Married,Unknown,Medium,4.51,1,0.0,0.0,1
29,73816,0.17,3.40,6.94,1.05,223.91,250.00,2.10,0.50,3.90,...,1,Short Term (< 10 Years),Married,NotApplicable,Low,1.54,1,0.0,0.0,1
32,73848,0.28,1.45,9.85,7.06,193.52,183.00,1.20,0.50,3.80,...,1,Unknown,Separated/Divorced,Unknown,Low,1.03,1,0.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9839,83590,0.78,1.62,11.02,2.34,207.51,222.00,2.17,0.54,3.93,...,1,Unknown,Married,NotApplicable,Medium,3.87,1,0.0,0.0,1
9854,83623,0.78,1.62,11.00,2.34,207.46,222.00,2.16,0.54,3.93,...,1,Unknown,Married,NotApplicable,Low,1.67,1,0.0,1.0,1
9868,83660,0.79,1.62,11.23,2.41,205.53,222.57,2.18,0.54,3.93,...,1,Unknown,Married,NotApplicable,High,5.00,1,0.0,1.0,1
9870,83663,0.79,1.62,11.23,2.41,205.53,222.57,2.20,0.54,3.93,...,1,Unknown,Separated/Divorced,Unknown,Low,1.13,1,0.0,0.0,1


In [13]:
df_final.to_csv('final_data_cleaned_17nov.csv')